In [141]:
import numpy as np
import matplotlib.pyplot as plt
import pyhmc as phmc
import scipy.linalg as la

import lmfit

from iminuit import Minuit

from pprint import pprint

from scipy.optimize import curve_fit

Nt=16
n_smear = 5
n_op = 4

def corr_th(x, a,b,c,d):
    #E= np.arccosh(2 - np.cos(2*np.pi/Nt))
    return a * np.cosh((x-Nt/2)*b) + c*np.cosh((x-Nt/2)*d)

def fit_bootstrap(p0, datax, datay, function, yerr_systematic=0.0):

    errfunc = lambda p, x, y: function(x,p) - y

    # Fit first time
    pfit, perr = optimize.leastsq(errfunc, p0, args=(datax, datay), full_output=0)


    # Get the stdev of the residuals
    residuals = errfunc(pfit, datax, datay)
    sigma_res = np.std(residuals)

    sigma_err_total = np.sqrt(sigma_res**2 + yerr_systematic**2)

    # 100 random data sets are generated and fitted
    ps = []
    for i in range(100):

        randomDelta = np.random.normal(0., sigma_err_total, len(datay))
        randomdataY = datay + randomDelta

        randomfit, randomcov = \
            optimize.leastsq(errfunc, p0, args=(datax, randomdataY),\
                             full_output=0)

        ps.append(randomfit) 

    ps = np.array(ps)
    mean_pfit = np.mean(ps,0)

    # You can choose the confidence interval that you want for your
    # parameter estimates: 
    Nsigma = 1. # 1sigma gets approximately the same as methods above
                # 1sigma corresponds to 68.3% confidence interval
                # 2sigma corresponds to 95.44% confidence interval
    err_pfit = Nsigma * np.std(ps,0) 

    pfit_bootstrap = mean_pfit
    perr_bootstrap = err_pfit
    return pfit_bootstrap, perr_bootstrap 

def variational_analysis4_fit(kappa,eigen, eigen_plus,eigen_minus,n_s):

    avg = [0 for i in range(4)]
    plus = [0 for i in range(4)]
    minus = [0 for i in range(4)]
    C=[0 for t in range(Nt)]
    C_plus=[0 for t in range(Nt)]
    C_minus=[0 for t in range(Nt)]
    T= list(range(Nt))
    index =[n_s,n_s+n_smear,n_s+n_smear*2,n_s+n_smear*3]
    c_mat = [[0 for i in range(4) ]for j in range(4) ]
    c_mat0 = [[0 for i in range(4) ]for j in range(4) ]
    c_plus = [[0 for i in range(4) ]for j in range(4) ]
    c_minus = [[0 for i in range(4) ]for j in range(4) ]
    avg_np= np.loadtxt('cross/matrix_np_L%d_k%f.txt' %(Nt,kappa))
    avg_np=avg_np.reshape(Nt,n_smear*n_op,n_smear*n_op)
    err_np= np.loadtxt('cross/matrix_np_L%d_k%f_err.txt' %(Nt,kappa))
    err_np=err_np.reshape(Nt,n_smear*n_op,n_smear*n_op)
    n_eigen=4
    
    for t in range(Nt):
        eigen.append([])
        eigen_plus.append([])
        eigen_minus.append([])
        for i in range(4):
            for j in range(4):
                if t == 0:
                    #c_mat0[i][j] = c_mat[0][i][j][t]
                    c_mat0[i][j] = avg_np[t][index[i]][index[j]]
                    #c_plus0[i][j] = c_res[0][i][j][t] + c_res[1][i][j][t]
                    #c_minus0[i][j] = c_res[0][i][j][t] - c_res[1][i][j][t]
                c_mat[i][j] = avg_np[t][index[i]][index[j]]
                #c_mat_err[i][j] = c_res[1][i][j][t]
                c_plus[i][j] = avg_np[t][index[i]][index[j]] + err_np[t][index[i]][index[j]]
                c_minus[i][j] = avg_np[t][index[i]][index[j]] - err_np[t][index[i]][index[j]]
                
        v,w = la.eig(c_mat,b=c_mat0)
        p,l = la.eig(c_plus,b=c_mat0)
        m,r = la.eig(c_minus,b=c_mat0)
        if t>0:
            v[::-1].sort()
            p[::-1].sort()
            m[::-1].sort()
        for i in range(4):
            if p[i].real > v[i].real and p[i].real > m[i].real:
                if v[i].real > m[i].real:
                    avg[i] = v[i].real
                    plus[i] = p[i].real
                    minus[i] = m[i].real
                if m[i].real > v[i].real:
                    avg[i] = m[i].real
                    plus[i] = p[i].real
                    minus[i] = v[i].real
            if v[i].real > p[i].real and v[i].real > m[i].real:
                if p[i].real > m[i].real:
                    avg[i] = p[i].real
                    plus[i] = v[i].real
                    minus[i] = m[i].real
                if m[i].real > p[i].real:
                    avg[i] = m[i].real
                    plus[i] = v[i].real
                    minus[i] = p[i].real
            if m[i].real > v[i].real and m[i].real > p[i].real:
                if v[i].real > p[i].real:
                    avg[i] = v[i].real
                    plus[i] = m[i].real
                    minus[i] = p[i].real
                if p[i].real > v[i].real:
                    avg[i] = p[i].real
                    plus[i] = m[i].real 
                    minus[i] = v[i].real
            eigen[t].append(avg[i])
            eigen_plus[t].append(plus[i])
            eigen_minus[t].append(minus[i])
            
    for ns in range(n_eigen):
        print("Eigenvalue level: ",ns)
        for t in range(Nt):
            C[t] = float(eigen[t][ns])
            C_plus[t] = float(eigen_plus[t][ns])
            C_minus[t] = float(eigen_minus[t][ns])
        #print(C)
            
        try:
            popt, pcov = curve_fit(corr_th, T, C,bounds =([0.0001,0,0,0],[4.0,2,1,6]))
            popt_plus, pcov = curve_fit(corr_th, T, C_plus,bounds =([0.0001,0,0,0],[4.0,2,1,6]))
            popt_minus, pcov = curve_fit(corr_th, T, C_minus,bounds =([0.0001,0,0,0],[4.0,2,1,6]))
            #print(popt)
            #print(popt_plus)
            #print(popt_minus)
            Energy=popt[1]
            En_err=np.abs((popt_plus[1]- popt_minus[1])/2)
            En2=popt[3]
            En2_err=np.abs((popt_plus[3]- popt_minus[3])/2)
            print('Energy = ',Energy,' \pm ',En_err )
            print('Energy2 = ',En2,' \pm ',En2_err )

        except RuntimeError:
            print("Error - curve_fit failed")
            
def variational_analysis4_cov(kappa,eigen, eigen_err,n_s):
    avg = [0 for i in range(4)]
    err = [0 for i in range(4)]
    C=[0 for t in range(Nt)]
    C_err=[0 for t in range(Nt)]
    T=list(range(Nt))
    index =[n_s,n_s+n_smear,n_s+n_smear*2,n_s+n_smear*3]
    c_mat = [[0 for i in range(4) ]for j in range(4) ]
    c_mat0 = [[0 for i in range(4) ]for j in range(4) ]
    c_plus = [[0 for i in range(4) ]for j in range(4) ]
    c_minus = [[0 for i in range(4) ]for j in range(4) ]
    avg_np= np.loadtxt('cross/matrix_run50_np_L%d_k%f.txt' %(Nt,kappa))
    avg_np=avg_np.reshape(Nt,n_smear*n_op,n_smear*n_op)
    err_np= np.loadtxt('cross/matrix_run50_np_L%d_k%f_err.txt' %(Nt,kappa))
    err_np=err_np.reshape(Nt,n_smear*n_op,n_smear*n_op)
    n_eigen=4
    
    for t in range(Nt):
        eigen.append([])
        eigen_err.append([])
        for i in range(4):
            for j in range(4):
                if t == 0:
                    c_mat0[i][j] = avg_np[t][index[i]][index[j]]
                c_mat[i][j] = avg_np[t][index[i]][index[j]]
                c_plus[i][j] = avg_np[t][index[i]][index[j]] + err_np[t][index[i]][index[j]]
                c_minus[i][j] = avg_np[t][index[i]][index[j]] - err_np[t][index[i]][index[j]]
                
        v,w = la.eig(c_mat,b=c_mat0)
        p,l = la.eig(c_plus,b=c_mat0)
        m,r = la.eig(c_minus,b=c_mat0)
        if t>0:
            v[::-1].sort()
            p[::-1].sort()
            m[::-1].sort()
        #print(v)
        #print(p)
        #print(m)
        for i in range(4):
            if p[i].real > v[i].real and p[i].real > m[i].real:
                if v[i].real > m[i].real:
                    avg[i] = v[i].real
                    err[i] = (p[i].real - m[i].real)/2
                if m[i].real > v[i].real:
                    avg[i] = m[i].real
                    err[i] = (p[i].real -v[i].real)/2
            if v[i].real > p[i].real and v[i].real > m[i].real:
                if p[i].real > m[i].real:
                    avg[i] = p[i].real
                    err[i] = (v[i].real - m[i].real)/2
                if m[i].real > p[i].real:
                    avg[i] = m[i].real
                    err[i] = (v[i].real -p[i].real)/2
            if m[i].real > v[i].real and m[i].real > p[i].real:
                if v[i].real > p[i].real:
                    avg[i] = v[i].real
                    err[i] = (m[i].real - p[i].real)/2
                if p[i].real > v[i].real:
                    avg[i] = p[i].real
                    err[i] = (m[i].real - v[i].real)/2
            eigen[t].append(avg[i])
            eigen_err[t].append(err[i])

    for ns in range(n_eigen):
        print("Eigenvalue level: ",ns)
        for t in range(Nt):
            C[t] = float(eigen[t][ns])
            C_err[t] = float(eigen_err[t][ns])

        #print(C)
            
        try:
            popt, pcov = curve_fit(corr_th, T, C,sigma = C_err, absolute_sigma = True,bounds =([0.0001,0,0,0],[4.0,2,1,6]))
            Energy=popt[1]
            En_err=np.sqrt(pcov[1][1])
            En2=popt[3]
            En2_err=np.sqrt(pcov[3][3])
            print('Energy = ',Energy,' \pm ',En_err )
            print('Energy2 = ',En2,' \pm ',En2_err )

        except RuntimeError:
            print("Error - curve_fit failed")
            
def variational_analysis4_lm(kappa,eigen, eigen_err,n_s):
    avg = [0 for i in range(4)]
    err = [0 for i in range(4)]
    C=[0 for t in range(Nt)]
    C_err=[0 for t in range(Nt)]
    T=list(range(Nt))
    index =[n_s,n_s+n_smear,n_s+n_smear*2,n_s+n_smear*3]
    c_mat = [[0 for i in range(4) ]for j in range(4) ]
    c_mat0 = [[0 for i in range(4) ]for j in range(4) ]
    c_plus = [[0 for i in range(4) ]for j in range(4) ]
    c_minus = [[0 for i in range(4) ]for j in range(4) ]
    avg_np= np.loadtxt('cross/matrix_np_L%d_k%f.txt' %(Nt,kappa))
    avg_np=avg_np.reshape(Nt,n_smear*n_op,n_smear*n_op)
    err_np= np.loadtxt('cross/matrix_np_L%d_k%f_err.txt' %(Nt,kappa))
    err_np=err_np.reshape(Nt,n_smear*n_op,n_smear*n_op)
    n_eigen=4
    
    for t in range(Nt):
        eigen.append([])
        eigen_err.append([])
        for i in range(4):
            for j in range(4):
                if t == 0:
                    c_mat0[i][j] = avg_np[t][index[i]][index[j]]
                c_mat[i][j] = avg_np[t][index[i]][index[j]]
                c_plus[i][j] = avg_np[t][index[i]][index[j]] + err_np[t][index[i]][index[j]]
                c_minus[i][j] = avg_np[t][index[i]][index[j]] - err_np[t][index[i]][index[j]]
                
        v,w = la.eig(c_mat,b=c_mat0)
        p,l = la.eig(c_plus,b=c_mat0)
        m,r = la.eig(c_minus,b=c_mat0)
        if t>0:
            v[::-1].sort()
            p[::-1].sort()
            m[::-1].sort()
        #print(v)
        #print(p)
        #print(m)
        for i in range(4):
            if p[i].real > v[i].real and p[i].real > m[i].real:
                if v[i].real > m[i].real:
                    avg[i] = v[i].real
                    err[i] = (p[i].real - m[i].real)/2
                if m[i].real > v[i].real:
                    avg[i] = m[i].real
                    err[i] = (p[i].real -v[i].real)/2
            if v[i].real > p[i].real and v[i].real > m[i].real:
                if p[i].real > m[i].real:
                    avg[i] = p[i].real
                    err[i] = (v[i].real - m[i].real)/2
                if m[i].real > p[i].real:
                    avg[i] = m[i].real
                    err[i] = (v[i].real -p[i].real)/2
            if m[i].real > v[i].real and m[i].real > p[i].real:
                if v[i].real > p[i].real:
                    avg[i] = v[i].real
                    err[i] = (m[i].real - p[i].real)/2
                if p[i].real > v[i].real:
                    avg[i] = p[i].real
                    err[i] = (m[i].real - v[i].real)/2
            eigen[t].append(avg[i])
            eigen_err[t].append(err[i])
            
    pars = lmfit.Parameters()
    pars.add_many(('a1', 1), ('E1', 0.5),('a2',0.1),('E2',0.5))
    
    t= np.linspace(0,Nt-1,Nt-1)


    for ns in range(n_eigen):
        print("Eigenvalue level: ",ns)
        for t in range(Nt):
            C[t] = float(eigen[t][ns])
            C_err[t] = float(eigen_err[t][ns])
        C = np.array(C)
        C_err= np.array(C_err)
        
        
        def residual(p):
            return (p['a1'] * np.cosh((t-Nt/2)*p['E1']) + p['a2']*np.cosh((t-Nt/2)*p['E2']) -C)/(C_err)
            
    
        mini = lmfit.Minimizer(residual, pars)
        try:
            result = mini.minimize()
            print(lmfit.fit_report(result.params))
        except ValueError:
            print("Error - lm_fit failed")
            
def variational_analysis4_min(kappa,eigen, eigen_err,n_s):
    f1=open('fit_results_4.dat','a+')
    print('Length = '+str(Nt)+'\n',file = f1)
    print('Kappa = '+str(k)+'\n',file=f1)
    print('Smear level = '+str(sm)+'\n',file=f1)
    avg = [0 for i in range(4)]
    err = [0 for i in range(4)]
    C=[0 for t in range(Nt)]
    C_err=[0 for t in range(Nt)]
    T=list(range(Nt))
    index =[n_s,n_s+n_smear,n_s+n_smear*2,n_s+n_smear*3]
    c_mat = [[0 for i in range(4) ]for j in range(4) ]
    c_mat0 = [[0 for i in range(4) ]for j in range(4) ]
    c_plus = [[0 for i in range(4) ]for j in range(4) ]
    c_minus = [[0 for i in range(4) ]for j in range(4) ]
    avg_np= np.loadtxt('cross/matrix_np_L%d_k%f.txt' %(Nt,kappa))
    avg_np=avg_np.reshape(Nt,n_smear*n_op,n_smear*n_op)
    err_np= np.loadtxt('cross/matrix_np_L%d_k%f_err.txt' %(Nt,kappa))
    err_np=err_np.reshape(Nt,n_smear*n_op,n_smear*n_op)
    n_eigen=4
    
    for t in range(Nt):
        eigen.append([])
        eigen_err.append([])
        for i in range(4):
            for j in range(4):
                if t == 0:
                    c_mat0[i][j] = avg_np[t][index[i]][index[j]]
                c_mat[i][j] = avg_np[t][index[i]][index[j]]
                c_plus[i][j] = avg_np[t][index[i]][index[j]] + err_np[t][index[i]][index[j]]
                c_minus[i][j] = avg_np[t][index[i]][index[j]] - err_np[t][index[i]][index[j]]
                
        v,w = la.eig(c_mat,b=c_mat0)
        p,l = la.eig(c_plus,b=c_mat0)
        m,r = la.eig(c_minus,b=c_mat0)
        if t>0:
            v[::-1].sort()
            p[::-1].sort()
            m[::-1].sort()
        #print(v)
        #print(p)
        #print(m)
        for i in range(4):
            if p[i].real > v[i].real and p[i].real > m[i].real:
                if v[i].real > m[i].real:
                    avg[i] = v[i].real
                    err[i] = (p[i].real - m[i].real)/2
                if m[i].real > v[i].real:
                    avg[i] = m[i].real
                    err[i] = (p[i].real -v[i].real)/2
            if v[i].real > p[i].real and v[i].real > m[i].real:
                if p[i].real > m[i].real:
                    avg[i] = p[i].real
                    err[i] = (v[i].real - m[i].real)/2
                if m[i].real > p[i].real:
                    avg[i] = m[i].real
                    err[i] = (v[i].real -p[i].real)/2
            if m[i].real > v[i].real and m[i].real > p[i].real:
                if v[i].real > p[i].real:
                    avg[i] = v[i].real
                    err[i] = (m[i].real - p[i].real)/2
                if p[i].real > v[i].real:
                    avg[i] = p[i].real
                    err[i] = (m[i].real - v[i].real)/2
            eigen[t].append(avg[i])
            eigen_err[t].append(err[i])

    for ns in range(n_eigen):
        print("Eigenvalue level: "+str(ns)+"\n",file=f1)
        for t in range(Nt):
            C[t] = float(eigen[t][ns])
            C_err[t] = float(eigen_err[t][ns])
            
        C=np.array(C)
        C_err=np.array(C_err)

        def least_squares(a, b,c,d):
            t=np.linspace(0,Nt-1,Nt)
            return sum((C - corr_th(t, a, b,c,d)) ** 2 / (C_err*C_err))


        try:
            #popt, pcov = curve_fit(corr_th, T, C,sigma = C_err, absolute_sigma = True,bounds =([0.0001,0,0,0],[4.0,2,1,6]))
            m = Minuit(least_squares,a=1,b=0.5,c=0.1,d=1,error_a=0.1,error_b=0.05,error_c=0.01,error_d=0.1,limit_a=(0,10),limit_b=(0,2),limit_c=(0,10),limit_d=(0,5),errordef=1)
            m.migrad()
            #print(m.np_values(),file=f1)
            #print(m.np_errors(),file=f1)
            E=m.np_values()[1]
            En_err=m.np_errors()[1]
            En2=m.np_values()[3]
            En2_err=m.np_errors()[3]
            print('Energy = ',E,' \pm ',En_err ,file=f1)
            print('Energy2 = ',En2,' \pm ',En2_err,file=f1 )

        except RuntimeError:
            print("Error - curve_fit failed")
    f1.close()
    
def variational_analysis3_min(kappa,eigen, eigen_err,n_s,n2):
    f1=open('fit_results_3.dat','a+')
    print('Length = '+str(Nt)+'\n',file = f1)
    print('Kappa = '+str(k)+'\n',file=f1)
    print('Smear level = '+str(sm)+'\n',file=f1)
    print('Operator excluded = '+str(n2)+'\n',file=f1)
    avg = [0 for i in range(4)]
    err = [0 for i in range(4)]
    index =[n_s,n_s+5,n_s+10,n_s+15]
    c_mat0 = [[0 for i in range(4) ]for j in range(4) ]
    avg_np= np.loadtxt('cross/matrix_np_L%d_k%f.txt' %(Nt,kappa))
    avg_np=avg_np.reshape(Nt,n_smear*n_op,n_smear*n_op)
    err_np= np.loadtxt('cross/matrix_np_L%d_k%f_err.txt' %(Nt,kappa))
    err_np=err_np.reshape(Nt,n_smear*n_op,n_smear*n_op)
    
    n_eigen=3
    
    C=[0 for t in range(Nt)]
    C_err=[0 for t in range(Nt)]
    T=list(range(Nt))
    
    for t in range(Nt):
        c_mat = [[0 for i in range(4) ]for j in range(4) ]
        c_plus = [[0 for i in range(4) ]for j in range(4) ]
        c_minus = [[0 for i in range(4) ]for j in range(4) ]
        eigen.append([])
        eigen_err.append([])
        for i in range(4):
            for j in range(4):
                if t == 0:
                    c_mat0[i][j] = avg_np[t][index[i]][index[j]]
                c_mat[i][j] = avg_np[t][index[i]][index[j]]
                c_plus[i][j] = avg_np[t][index[i]][index[j]] + err_np[t][index[i]][index[j]]
                c_minus[i][j] = avg_np[t][index[i]][index[j]] - err_np[t][index[i]][index[j]]

        [j.pop(n2) for j in c_mat]
        c_mat.pop(n2)
        if t == 0:
            [j.pop(n2) for j in c_mat0]
            c_mat0.pop(n2)
        [j.pop(n2) for j in c_plus]
        c_plus.pop(n2)
        [j.pop(n2) for j in c_minus]
        c_minus.pop(n2)
        v,w = la.eig(c_mat,b=c_mat0)
        p,l = la.eig(c_plus,b=c_mat0)
        m,r = la.eig(c_minus,b=c_mat0)
        if t >0:
            v[::-1].sort()
            p[::-1].sort()
            m[::-1].sort()
        for i in range(3):
            if p[i].real > v[i].real and p[i].real > m[i].real:
                if v[i].real > m[i].real:
                    avg[i] = v[i].real
                    err[i] = (p[i].real - m[i].real)/2
                if m[i].real > v[i].real:
                    avg[i] = m[i].real
                    err[i] = (p[i].real -v[i].real)/2
            if v[i].real > p[i].real and v[i].real > m[i].real:
                if p[i].real > m[i].real:
                    avg[i] = p[i].real
                    err[i] = (v[i].real - m[i].real)/2
                if m[i].real > p[i].real:
                    avg[i] = m[i].real
                    err[i] = (v[i].real -p[i].real)/2
            if m[i].real > v[i].real and m[i].real > p[i].real:
                if v[i].real > p[i].real:
                    avg[i] = v[i].real
                    err[i] = (m[i].real - p[i].real)/2
                if p[i].real > v[i].real:
                    avg[i] = p[i].real
                    err[i] = (m[i].real - v[i].real)/2
            eigen[t].append(avg[i])
            eigen_err[t].append(err[i])
        #print('Eigenvalue problem solved without the operator',n2)
        #print(t,'Eigenvalues: ',eigen[t], 'Errors: ',eigen_err[t])

    for ns in range(n_eigen):
        print("Eigenvalue level: "+str(ns)+"\n",file=f1)
        for t in range(Nt):
            C[t] = float(eigen[t][ns])
            C_err[t] = float(eigen_err[t][ns])
            
        C=np.array(C)
        C_err=np.array(C_err)

        def least_squares(a, b,c,d):
            t=np.linspace(0,Nt-1,Nt)
            return sum((C - corr_th(t, a, b,c,d)) ** 2 / (C_err*C_err))
 
        try:
            m = Minuit(least_squares,a=1,b=0.5,c=0.1,d=1,error_a=0.1,error_b=0.05,error_c=0.01,error_d=0.1,limit_a=(0,10),limit_b=(0,2),limit_c=(0,10),limit_d=(0,5),errordef=1)
            m.migrad()
            print(m.np_values(),file=f1)
            print(m.np_errors(),file=f1)

        except RuntimeError:
            print("Error - curve_fit failed")
    f1.close()
    
def var_analysis_4_fit(kappa,n_s):
    Eigen =[]
    Eigen_plus=[]
    Eigen_minus=[]
    variational_analysis4_fit(kappa,Eigen,Eigen_plus,Eigen_minus,n_s)
    
def var_analysis_4_cov(kappa,n_s):
    Eigen =[]
    Eigen_err=[]
    variational_analysis4_cov(kappa,Eigen,Eigen_err,n_s)
    
def var_analysis_4_lm(kappa,n_s):
    Eigen =[]
    Eigen_err=[]
    variational_analysis4_lm(kappa,Eigen,Eigen_err,n_s)
    
def var_analysis_4_min(kappa,n_s):
    Eigen =[]
    Eigen_err=[]
    variational_analysis4_min(kappa,Eigen,Eigen_err,n_s)
    
def var_analysis_3_min(kappa,n_s,n2):
    Eigen =[]
    Eigen_err=[]
    variational_analysis3_min(kappa,Eigen,Eigen_err,n_s,n2)



In [76]:
k_l=[0.55,0.65]
Nt_l= [8,16,24,32]
for Nt in [16]:
    print('Length = ',Nt)
    for k in [0.55]:
        print('Kappa = ',k)
        for sm in range(n_smear):
            print('Smear level = ', sm)
            var_analysis_plot4_fit(k,sm)

Length =  16
Kappa =  0.55
Smear level =  0
Eigenvalue level:  0
Energy =  0.38981666018949734  \pm  0.01720941634401743
Energy2 =  2.4063968933457094  \pm  0.0626421554934109
Eigenvalue level:  1
Energy =  0.5916245113400987  \pm  0.15876934151615352
Energy2 =  1.618306769576848  \pm  0.5740448815862048
Eigenvalue level:  2
Error - curve_fit failed
Eigenvalue level:  3
Error - curve_fit failed
Smear level =  1
Eigenvalue level:  0
Energy =  0.38640202682955554  \pm  0.0171810343208956
Energy2 =  2.304585212792766  \pm  0.012991356537311649
Eigenvalue level:  1
Energy =  0.4891934344627368  \pm  0.18337939138403456
Energy2 =  1.4402076048665373  \pm  0.43632379816571953
Eigenvalue level:  2
Error - curve_fit failed
Eigenvalue level:  3
Error - curve_fit failed
Smear level =  2
Eigenvalue level:  0
Energy =  0.3868113814008693  \pm  0.01658419417292259
Energy2 =  2.3444019394042237  \pm  0.04694733132136619
Eigenvalue level:  1
Error - curve_fit failed
Eigenvalue level:  2
Error - curve

In [26]:
k_l=[0.55,0.65]
Nt_l= [8,16,24,32]
for Nt in [32]:
    print('Length = ',Nt)
    for k in [0.65]:
        print('Kappa = ',k)
        for sm in range(n_smear):
            print('Smear level = ', sm)
            var_analysis_plot4_cov(k,sm)

Length =  32
Kappa =  0.65
Smear level =  0
Eigenvalue level:  0
Error - curve_fit failed
Eigenvalue level:  1
Energy =  0.08605058230422795  \pm  0.1049401639234558
Energy2 =  0.44271252114238646  \pm  0.026385141896585907
Eigenvalue level:  2
Error - curve_fit failed
Eigenvalue level:  3
Energy =  0.5670010082101234  \pm  0.011653341081373971
Energy2 =  1.2755513895172654e-05  \pm  0.0
Smear level =  1
Eigenvalue level:  0
Error - curve_fit failed
Eigenvalue level:  1
Error - curve_fit failed
Eigenvalue level:  2
Error - curve_fit failed
Eigenvalue level:  3
Error - curve_fit failed
Smear level =  2
Eigenvalue level:  0
Energy =  0.4945268226504391  \pm  0.2869067919322591
Energy2 =  0.18463051386995583  \pm  0.01229917359136774
Eigenvalue level:  1
Error - curve_fit failed
Eigenvalue level:  2
Error - curve_fit failed
Eigenvalue level:  3
Error - curve_fit failed
Smear level =  3
Eigenvalue level:  0
Error - curve_fit failed
Eigenvalue level:  1
Energy =  0.3693883915396542  \pm  1.

In [36]:
Nt=16
var_analysis_plot4_lm(0.55,0)

Eigenvalue level:  0
[[Variables]]
    a1:  0.30008232 +/- 289456.731 (96459109.95%) (init = 1)
    E1:  0.42607071 +/- 229140.141 (53779838.68%) (init = 0.5)
    a2: -0.23847149 +/-        nan (nan%) (init = 0.1)
    E2:  0.44982606 +/- 372800.832 (82876664.37%) (init = 0.5)
[[Correlations]] (unreported correlations are < 0.100)
    C(a1, E1) =  3.341
    C(E1, E2) =  1.726
    C(a1, E2) =  1.558
Eigenvalue level:  1
Error - lm_fit failed
Eigenvalue level:  2
Error - lm_fit failed
Eigenvalue level:  3
Error - lm_fit failed


In [142]:
f= open('fit_results_4.dat','w+')
k_l=[0.55,0.65]
Nt_l= [8,16,24,32]
for Nt in Nt_l:
    for k in k_l:
        for sm in range(n_smear):        
            var_analysis_4_min(k,sm)
f.close()

In [137]:
k_l=[0.55,0.65]
Nt_l= [8,16,24,32]
f= open('fit_results_3.dat','w+')
for Nt in [16]:
    for k in k_l:
        for sm in range(n_smear):
            for op in range(n_op):
                var_analysis_3_min(k,sm,op)
f.close()

Eigenvalue problem solved without the operator 0
0 Eigenvalues:  [1.0000000000000002, 0.9999999999999999, 1.0] Errors:  [0.04792397917285418, 0.025768072682690435, 0.0067184708834886475]
Eigenvalue problem solved without the operator 0
1 Eigenvalues:  [0.3523639671296276, 0.24741151394919833, 0.09192472359335005] Errors:  [0.007136541060603241, 0.015394729433757526, 0.01195894628506474]
Eigenvalue problem solved without the operator 0
2 Eigenvalues:  [0.1605526151302312, 0.14557549373911288, 0.045596769273045935] Errors:  [0.00939094642435015, 0.008512491455293217, 0.009898842183009966]
Eigenvalue problem solved without the operator 0
3 Eigenvalues:  [0.09926546210052464, 0.07480847469607249, 0.024237901319052513] Errors:  [0.010616569599797274, 0.004934570872016285, 0.009127742159683058]
Eigenvalue problem solved without the operator 0
4 Eigenvalues:  [0.06117423603059136, 0.03893441133626145, 0.012061395016362542] Errors:  [0.009104422363155062, 0.006124903571979055, 0.00732012109395

Eigenvalue problem solved without the operator 3
0 Eigenvalues:  [1.0000000000000002, 1.0, 1.0000000000000004] Errors:  [0.05867711525729946, 0.005421962506523548, 0.028129474763121554]
Eigenvalue problem solved without the operator 3
1 Eigenvalues:  [0.4926231637348641, 0.09803490066919084, 0.08836365304791348] Errors:  [0.009381944361980171, 0.013825613062430811, 0.011289834263858274]
Eigenvalue problem solved without the operator 3
2 Eigenvalues:  [0.3190758271016039, 0.057708555874256655, 0.027943405948598312] Errors:  [0.008528722951854334, 0.012815314004631994, 0.006210878046486318]
Eigenvalue problem solved without the operator 3
3 Eigenvalues:  [0.21676119624317758, 0.038539859964169164, 0.007503164574332446] Errors:  [0.00805370798095463, 0.012192573723845876, 0.004659329271236588]
Eigenvalue problem solved without the operator 3
4 Eigenvalues:  [0.14866203260452257, 0.02212301730177934, -0.0010674722842506173] Errors:  [0.008049313439382935, 0.012342851701491325, 0.0030248472

Eigenvalue problem solved without the operator 2
0 Eigenvalues:  [1.0000000000000004, 1.0000000000000002, 1.0000000000000002] Errors:  [0.03139841522951137, 0.0023027183781038074, 0.014051518873013202]
Eigenvalue problem solved without the operator 2
1 Eigenvalues:  [0.49156462891281555, 0.35127530065619716, 0.09932384098761124] Errors:  [0.010346618344595465, 0.0073591503134553815, 0.013342815662028558]
Eigenvalue problem solved without the operator 2
2 Eigenvalues:  [0.3190833584192241, 0.1622153863997763, 0.05847453117857135] Errors:  [0.009066939396859375, 0.006449630097185702, 0.012587674475245194]
Eigenvalue problem solved without the operator 2
3 Eigenvalues:  [0.2143336104418145, 0.07651651583477474, 0.03254086189839416] Errors:  [0.00845181039080517, 0.005975133933827544, 0.012206897724827815]
Eigenvalue problem solved without the operator 2
4 Eigenvalues:  [0.14919352933653424, 0.04154792439717255, 0.03214496979881507] Errors:  [0.008170589114430166, 0.005379255483524838, 0.0

Eigenvalue problem solved without the operator 1
0 Eigenvalues:  [1.0000000000000002, 0.9999999999999996, 1.0000000000000002] Errors:  [0.05620674468868325, 0.0011438498338205982, 0.013276276189414349]
Eigenvalue problem solved without the operator 1
1 Eigenvalues:  [0.44017239951725384, 0.34897255892477896, 0.09004967379590054] Errors:  [0.008212868400517576, 0.007519781962574329, 0.01110747606808947]
Eigenvalue problem solved without the operator 1
2 Eigenvalues:  [0.28432460491536155, 0.16063931958679975, 0.026237979339582253] Errors:  [0.008048057412489956, 0.006570744725521424, 0.007421076004218371]
Eigenvalue problem solved without the operator 1
3 Eigenvalues:  [0.1906716795159037, 0.0769376100453537, 0.008474279625524914] Errors:  [0.007866456291585791, 0.006205001984215121, 0.005248272962941118]
Eigenvalue problem solved without the operator 1
4 Eigenvalues:  [0.13047628639308131, 0.03792387819871556, 0.0016325855483115825] Errors:  [0.0076885970527986736, 0.005621268062314565

Eigenvalue problem solved without the operator 0
0 Eigenvalues:  [0.9999999999999996, 1.0000000000000002, 0.9999999999999999] Errors:  [0.052840612621113836, 0.025913523182262643, 0.006925583341626074]
Eigenvalue problem solved without the operator 0
1 Eigenvalues:  [0.3488849627715595, 0.2521062965395569, 0.0901211654935864] Errors:  [0.007386100391317207, 0.014274474725680497, 0.01093485275181303]
Eigenvalue problem solved without the operator 0
2 Eigenvalues:  [0.16132800260554364, 0.14130886856943523, 0.03489646246020632] Errors:  [0.00839220686839559, 0.008292196001230684, 0.008820440452569757]
Eigenvalue problem solved without the operator 0
3 Eigenvalues:  [0.09326972322150617, 0.07658836666273895, 0.014155438465221213] Errors:  [0.011161121118559665, 0.005951089468468951, 0.009582177847241575]
Eigenvalue problem solved without the operator 0
4 Eigenvalues:  [0.06964396745491606, 0.03975960992962535, 0.020108646083115557] Errors:  [0.009875577660589014, 0.004991829918192282, 0.0

...

Eigenvalue problem solved without the operator 3
0 Eigenvalues:  [1.0000000000000002, 0.9999999999999999, 1.0] Errors:  [0.004419570108804838, 0.04544558948172345, 0.029738642725179898]
Eigenvalue problem solved without the operator 3
1 Eigenvalues:  [0.5227200370337508, 0.17392999444396176, 0.12068797388750423] Errors:  [0.007236209147359229, 0.010104029580313603, 0.008625991434835556]
Eigenvalue problem solved without the operator 3
2 Eigenvalues:  [0.34354193122619553, 0.1102986459704343, 0.039393641750287506] Errors:  [0.006527291893267867, 0.008840497967162866, 0.005663849243863497]
Eigenvalue problem solved without the operator 3
3 Eigenvalues:  [0.23207540302572202, 0.06602140984497806, 0.012795519240210869] Errors:  [0.006238826961272925, 0.008134576532911157, 0.00249831586182641]
Eigenvalue problem solved without the operator 3
4 Eigenvalues:  [0.15936334491879, 0.050421600129725096, 0.0026078723974411805] Errors:  [0.006142196359721269, 0.008572568903024238, 0.004106253227014

Eigenvalue problem solved without the operator 2
0 Eigenvalues:  [1.0, 0.9999999999999998, 1.0000000000000002] Errors:  [0.031176869032163268, 0.002165403241846997, 0.011333275930994646]
Eigenvalue problem solved without the operator 2
1 Eigenvalues:  [0.5230146956174607, 0.37604982869510767, 0.16610502342332978] Errors:  [0.007783301982048463, 0.005619030769064448, 0.009897981915207674]
Eigenvalue problem solved without the operator 2
2 Eigenvalues:  [0.3441870742409657, 0.1723351318025806, 0.1030736879293401] Errors:  [0.007002962127902274, 0.004882544828095484, 0.008916137181409371]
Eigenvalue problem solved without the operator 2
3 Eigenvalues:  [0.23272535380668047, 0.08073297906078042, 0.06955577546157826] Errors:  [0.006580887039430078, 0.0039243567711313085, 0.008668056191318534]
Eigenvalue problem solved without the operator 2
4 Eigenvalues:  [0.15952754224028626, 0.0432676347468741, 0.03968669023700116] Errors:  [0.0062206692480482745, 0.006720712268241273, 0.0062305243843885

Eigenvalue problem solved without the operator 1
0 Eigenvalues:  [1.0, 1.0, 0.9999999999999999] Errors:  [0.04530154390156471, 0.0011792955428777674, 0.01034614028842018]
Eigenvalue problem solved without the operator 1
1 Eigenvalues:  [0.48792579047729207, 0.3762325403757478, 0.12455116791677232] Errors:  [0.0067896829108690104, 0.005627199634979818, 0.009932562803367448]
Eigenvalue problem solved without the operator 1
2 Eigenvalues:  [0.3194361373355201, 0.1739303315238099, 0.032645172438571554] Errors:  [0.006403810827973644, 0.004871397446949852, 0.004236472873903963]
Eigenvalue problem solved without the operator 1
3 Eigenvalues:  [0.2160195159493216, 0.07983824310268875, 0.004677350099160876] Errors:  [0.006248284173629881, 0.004718797844209463, 0.005566492492155686]
Eigenvalue problem solved without the operator 1
4 Eigenvalues:  [0.14825584843310466, 0.03975622516954647, 0.0013094925232012996] Errors:  [0.006230640769337409, 0.004801224372970373, 0.0015897325005868265]
Eigenva

Eigenvalue problem solved without the operator 0
0 Eigenvalues:  [1.0000000000000002, 1.0, 1.0000000000000002] Errors:  [0.04220231538032515, 0.022150794990581035, 0.005449384087747755]
Eigenvalue problem solved without the operator 0
1 Eigenvalues:  [0.37664620538440025, 0.32786039273478806, 0.15244692469616147] Errors:  [0.005955586416518943, 0.011196134107627431, 0.009745215187996314]
Eigenvalue problem solved without the operator 0
2 Eigenvalues:  [0.198509563772824, 0.17373411672539774, 0.0726131509505896] Errors:  [0.009407428848442367, 0.00419196141750898, 0.006499710246307876]
Eigenvalue problem solved without the operator 0
3 Eigenvalues:  [0.13728170723660382, 0.0800060497843965, 0.03983581526605131] Errors:  [0.007847273078758363, 0.004784387061850352, 0.005034795482580547]
Eigenvalue problem solved without the operator 0
4 Eigenvalues:  [0.09384864934928197, 0.03957115528549339, 0.025589980266555384] Errors:  [0.0074616102447633, 0.004737747833139471, 0.004547587005692801]


Eigenvalue problem solved without the operator 3
0 Eigenvalues:  [0.9999999999999998, 0.9999999999999999, 0.9999999999999997] Errors:  [0.004705795719524786, 0.048763719624680346, 0.030472385115550338]
Eigenvalue problem solved without the operator 3
1 Eigenvalues:  [0.5227832098392957, 0.17562695190590916, 0.12673110389991926] Errors:  [0.007299535569784299, 0.010507027362863297, 0.009720882329540576]
Eigenvalue problem solved without the operator 3
2 Eigenvalues:  [0.3429209609075211, 0.10516815519222346, 0.03299747687004965] Errors:  [0.0067902703379604246, 0.009049255146958507, 0.004457207423339982]
Eigenvalue problem solved without the operator 3
3 Eigenvalues:  [0.23217840653908953, 0.0747905104454972, 0.010953874543580355] Errors:  [0.006211033963490903, 0.008558487968391149, 0.002932810974737729]
Eigenvalue problem solved without the operator 3
4 Eigenvalues:  [0.15981879741891505, 0.05056518102976932, 0.0039371710613279745] Errors:  [0.0062092027266047595, 0.008538728509277865